# Example 12: Physical Engineering Audit (FAIR Metrics)
-----------------------------------------------------
This example focuses on the **Facts** layer of the diagnostic suite. 
Unlike Q-Scores (interpretation), FAIR metrics provide raw, physical measurements of the population's health.

| Metric | Physical Meaning |
| :--- | :--- |
| **CLOSENESS** | Raw distance distribution to the truth manifold |
| **COVERAGE** | Average distance from GT to population (IGD) |
| **GAP** | Max hole size on the manifold |
| **REGULARITY** | Deviation from ideal lattice spacing (Wasserstein) |
| **BALANCE** | Mass distribution bias (JS Divergence) |

In [ ]:
import mb_path
import numpy as np
from MoeaBench import mb

# Setup
mop = mb.mops.DTLZ2(M=3)
gt = mop.pf(n_points=500)

### 1. Simulated Pathology: Central Gap
We run NSGA-II to convergence and then manually remove the central third of the front to simulate a structural flaw.

In [ ]:
exp = mb.experiment()
exp.mop = mop
exp.moea = mb.moeas.NSGA2(population=100)
exp.run(generations=150)

# Removal of the center
objs = exp.last_pop.objectives
mask = (objs[:, 0] < 0.35) | (objs[:, 0] > 0.65)
exp.last_pop.objectives = objs[mask]

mb.view.topo_shape(exp, gt, 
                   title="Topology: Central Gap Check",
                   labels=["Gapped Population", "Optimal Front (GT)"])

### 2. Physical Insight (Closeness & Coverage)
Closeness represents the raw proximity distribution. Coverage (IGD) represents the overall span.

In [ ]:
# A. Closeness (Physical Proximity Distribution)
u_dist = mb.diagnostics.fair_closeness(exp, ground_truth=gt)
print(f"Mean Distance: {np.mean(u_dist):.4f} units")
print(f"Max Distance (95th): {np.percentile(u_dist, 95):.4f} units")

# B. Coverage Scalar
diag = mb.diagnostics.audit(exp, ground_truth=gt)
f_res = diag.fair_audit_res.metrics

f_res["COVERAGE"].report_show()

### 3. Consolidated Physics Audit
The full physical report integrates all engineering facts.

In [ ]:
mb.diagnostics.fair_audit(exp, ground_truth=gt).report_show()